# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [3]:
import torch as th

In [4]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [5]:
y = x + x

In [6]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [ ]:
#!pip install syft
!pip install syft==0.2.9 
import syft as sy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 433 kB 8.7 MB/s 
     |████████████████████████████████| 200 kB 51.6 MB/s 
     |████████████████████████████████| 66 kB 4.2 MB/s 
     |████████████████████████████████| 449 kB 53.3 MB/s 
     |████████████████████████████████| 4.0 MB 57.5 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |█████████▎                      | 218.0 MB 1.4 MB/s eta 0:06:15

In [ ]:
import syft as sy

In [ ]:
hook = sy.TorchHook(th)

In [ ]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.send(bob)

In [ ]:
bob._objects

{64852415206: tensor([1, 2, 3, 4, 5])}

In [ ]:
x.location

<VirtualWorker id:bob #objects:1>

In [ ]:
x.id_at_location

64852415206

In [ ]:
x.id

36045003156

In [ ]:
x.owner

<VirtualWorker id:me #objects:0>

In [ ]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [ ]:
x

(Wrapper)>[PointerTensor | me:36045003156 -> bob:64852415206]

In [ ]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [ ]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
alice = sy.VirtualWorker(hook, id="alice")

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x_ptr = x.send(bob, alice)

In [ ]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob, alice)

In [ ]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

# Lesson: Introducing Remote Arithmetic

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x

(Wrapper)>[PointerTensor | me:65489164314 -> bob:76237791741]

In [ ]:
y

(Wrapper)>[PointerTensor | me:34175752412 -> bob:32358754238]

In [ ]:
z = x + y

In [ ]:
z

(Wrapper)>[PointerTensor | me:85761195774 -> bob:67325675157]

In [ ]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [ ]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:93357797126 -> bob:58647446174]

In [ ]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [ ]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

(Wrapper)>[PointerTensor | me:80240957570 -> bob:16650892512]

In [ ]:
x = x.get()

In [ ]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [ ]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [ ]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [ ]:
for i in range(10):

    pred = input.mm(weights)

    loss = ((pred - target)**2).sum()

    loss.backward()

    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [ ]:
bob = bob.clear_objects()

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{64186599582: tensor([1, 2, 3, 4, 5])}

In [ ]:
del x

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{61824499317: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = "asdf"

In [ ]:
bob._objects

{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

(Wrapper)>[PointerTensor | me:74310270083 -> bob:60968457887]

In [ ]:
bob._objects

{60968457887: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = "asdf"

In [ ]:
bob._objects

{60968457887: tensor([1, 2, 3, 4, 5])}

In [ ]:
del x

In [ ]:
bob._objects

{60968457887: tensor([1, 2, 3, 4, 5])}

In [ ]:
bob = bob.clear_objects()
bob._objects

{}

In [ ]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{18918034451: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [ ]:
z = x + y

TensorsNotCollocatedException: ignored

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [ ]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.3780)
tensor(0.6807)
tensor(0.4289)
tensor(0.3173)
tensor(0.2394)
tensor(0.1813)
tensor(0.1375)
tensor(0.1044)
tensor(0.0794)
tensor(0.0604)
tensor(0.0460)
tensor(0.0351)
tensor(0.0267)
tensor(0.0204)
tensor(0.0156)
tensor(0.0119)
tensor(0.0091)
tensor(0.0069)
tensor(0.0053)
tensor(0.0040)


In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [ ]:
train()

tensor(0.3148, requires_grad=True)
tensor(0.5356, requires_grad=True)
tensor(0.1061, requires_grad=True)
tensor(0.2892, requires_grad=True)
tensor(0.0913, requires_grad=True)
tensor(0.1859, requires_grad=True)
tensor(0.0610, requires_grad=True)
tensor(0.1223, requires_grad=True)
tensor(0.0409, requires_grad=True)
tensor(0.0816, requires_grad=True)
tensor(0.0279, requires_grad=True)
tensor(0.0551, requires_grad=True)
tensor(0.0194, requires_grad=True)
tensor(0.0377, requires_grad=True)
tensor(0.0137, requires_grad=True)
tensor(0.0260, requires_grad=True)
tensor(0.0098, requires_grad=True)
tensor(0.0182, requires_grad=True)
tensor(0.0071, requires_grad=True)
tensor(0.0128, requires_grad=True)
tensor(0.0051, requires_grad=True)
tensor(0.0091, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0065, requires_grad=True)
tensor(0.0028, requires_grad=True)
tensor(0.0047, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0034, requires_grad=True)
tensor(0.0015, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

{6756162089: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{53803814348: (Wrapper)>[PointerTensor | alice:53803814348 -> bob:6756162089]}

In [ ]:
y = x + x

In [ ]:
y

(Wrapper)>[PointerTensor | me:42329538110 -> alice:64710027361]

In [ ]:
bob._objects

{6756162089: tensor([1, 2, 3, 4, 5]),
 86165705876: tensor([ 2,  4,  6,  8, 10])}

In [ ]:
alice._objects

{53803814348: (Wrapper)>[PointerTensor | alice:53803814348 -> bob:6756162089],
 64710027361: (Wrapper)>[PointerTensor | alice:64710027361 -> bob:86165705876]}

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{96697413406: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{6548794844: (Wrapper)>[PointerTensor | alice:6548794844 -> bob:96697413406]}

In [ ]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:6548794844 -> bob:96697413406]

In [ ]:
bob._objects

{96697413406: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{}

In [ ]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [ ]:
bob._objects

{}

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{77923506546: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{13758651321: (Wrapper)>[PointerTensor | alice:13758651321 -> bob:77923506546]}

In [ ]:
del x

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

{43399725491: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{}

In [ ]:
x.move(alice)

(Wrapper)>[PointerTensor | me:26901242282 -> alice:43399725491]

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{43399725491: tensor([1, 2, 3, 4, 5])}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

{47118072744: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{43399725491: tensor([1, 2, 3, 4, 5]),
 2158080987: (Wrapper)>[PointerTensor | alice:2158080987 -> bob:47118072744]}

In [ ]:
x.remote_get()

(Wrapper)>[PointerTensor | me:37475137066 -> alice:2158080987]

In [ ]:
bob._objects

{}

In [ ]:
alice._objects

{43399725491: tensor([1, 2, 3, 4, 5]), 2158080987: tensor([1, 2, 3, 4, 5])}

In [ ]:
x.move(bob)

(Wrapper)>[PointerTensor | me:37475137066 -> bob:2158080987]

In [ ]:
x

(Wrapper)>[PointerTensor | me:37475137066 -> bob:2158080987]

In [ ]:
bob._objects

{2158080987: tensor([1, 2, 3, 4, 5])}

In [ ]:
alice._objects

{43399725491: tensor([1, 2, 3, 4, 5])}